# SQL

In [ ]:
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus
import pandas as pd

# Параметры подключения
host = "194.35.48.21"
user = "master-user"
password = "64fjeObn./d,DP][xds"
database = "payments_yakassa"

# Создание движка
engine = create_engine(f"mysql+mysqldb://{user}:%s@{host}/{database}" % quote_plus(password))

# SQL-запрос
query = """
SELECT id, order_id, price, status, created_date, pack_id, pack_identifier_id, user_id, email, device, device_name,
       cancel_reason, is_refund, refund_amount, user_pack_id, platform_id, app_id, is_autopay, is_promo
FROM payments_yakassa
WHERE app_id IN (561, 582)
"""

# Загрузка данных в DataFrame
with engine.connect() as connection:
    df = pd.read_sql(text(query), connection)

print(f"Загружено {len(df)} строк")


# Неделя подсчет конверсии

In [6]:
import pandas as pd
import time
from datetime import datetime
import requests
from io import StringIO

# === Пути к файлам ===
PAYMENTS_PATH = 'export_(ReportPayments.2025-02-14 - 2025-06-15.).csv'

# === Авторизация и параметры AppMetrica ===
API_TOKEN = "y0__xDunbWlqveAAhianDcgvtvu8hI4Lgj1FE3Wx6z8be6gSyQ7sTrc4A"
APPLICATION_ID = "4661140"
DATE_SINCE = "2025-02-14 00:00:00"
DATE_UNTIL = datetime.now().strftime("%Y-%m-%d 00:00:00")

# === Классификация переходов ===
def classify_transactions(group):
    group = group.sort_values('дата')
    types = []
    promo_found = False
    full_found = False

    for _, row in group.iterrows():
        is_auto = str(row['тип_платежа']).strip().lower() == 'автоплатеж'
        is_promo = row['промо'] == 1

        if is_promo:
            types.append('promo')
            promo_found = True
        elif is_auto and promo_found and not full_found:
            types.append('full')
            full_found = True
        elif is_auto and full_found:
            types.append('repeat')
        else:
            types.append('other')

    group['тип_сделки'] = types
    return group


def fetch_installations_csv():
    url = (
        f"https://api.appmetrica.yandex.ru/logs/v1/export/installations.csv?"
        f"application_id={APPLICATION_ID}"
        f"&date_since={DATE_SINCE.replace(' ', '%20')}"
        f"&date_until={DATE_UNTIL.replace(' ', '%20')}"
        f"&date_dimension=default"
        f"&fields=tracker_name,profile_id"
    )
    headers = {"Authorization": f"OAuth {API_TOKEN}"}
    
    print("Отправка запроса в AppMetrica...")
    for attempt in range(30):
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            print("Данные получены.")
            df = pd.read_csv(StringIO(response.text), header=0)
            return df
        elif response.status_code == 202:
            print(f"Попытка {attempt + 1}/30: данные ещё не готовы, ждём ...")
            time.sleep(20)
        else:
            raise Exception(f"Ошибка: {response.status_code}, {response.text}")
    
    raise TimeoutError("Данные не были готовы в течение 5 минут.")


# === Загрузка платежей ===
with open(PAYMENTS_PATH, 'r', encoding='utf-8') as f:
    lines = f.readlines()

header_line = lines[10].strip().split(';')
df = pd.read_csv(PAYMENTS_PATH, skiprows=11, sep=';', names=header_line)
df.columns = [col.strip().lower().replace(' ', '_') for col in df.columns]
df['дата'] = pd.to_datetime(df['дата'], errors='coerce')

# === Фильтрация успешных платежей ===
df = df[
    (df['статус'].str.lower() == 'успешно') & 
    (df['e-mail_оплаты'].notnull())
].sort_values(by=['e-mail_оплаты', 'id_пакета', 'дата'])

df = df.groupby(['e-mail_оплаты', 'id_пакета']).apply(classify_transactions).reset_index(drop=True)

# === Повторные платежи ===
repeat_df = df[df['тип_сделки'] == 'repeat'].copy()
df = df[df['тип_сделки'] == 'full']

# === Загрузка трекеров через API ===
install_df = fetch_installations_csv()
install_df = install_df.dropna(subset=['profile_id'])
install_df = install_df.rename(columns={'profile_id': 'девайс_оплаты'})
install_df['tracker_name'] = install_df['tracker_name'].fillna('unknown')

df = df.merge(install_df, on='девайс_оплаты', how='left')
df['tracker_name'] = df['tracker_name'].fillna('unknown')
repeat_df = repeat_df.merge(install_df, on='девайс_оплаты', how='left')
repeat_df['tracker_name'] = repeat_df['tracker_name'].fillna('unknown')

# === Добавление года и номера недели ===
for target_df in [df, repeat_df]:
    calendar = target_df['дата'].dt.isocalendar()
    target_df['год'] = calendar.year
    target_df['неделя'] = calendar.week
    target_df['неделя_диапазон'] = target_df['дата'].dt.to_period('W').astype(str)

# === Создание строки пользователя ===
df['пользователь'] = df.apply(
    lambda row: f"{row['e-mail_оплаты']} — {row['девайс_оплаты']} — [{row['tracker_name']}] — {row['сумма']}₽", axis=1
)

# === Группировка по неделям и пакетам ===
grouped = (
    df.groupby(['год', 'неделя', 'неделя_диапазон', 'пакет'])
    .agg(переходы=('пользователь', lambda x: sorted(set(x))))
    .reset_index()
    .sort_values(by=['год', 'неделя'], ascending=False)
)

print("Переходы по неделям и пакетам:\n")
total = 0
for _, row in grouped.iterrows():
    count = len(row['переходы'])
    total += count
    print(f"{row['неделя_диапазон']} — {row['пакет']}: {count}")
print(f"\nВсего переходов: {total}\n")

for _, row in grouped.iterrows():
    print(f"{row['неделя_диапазон']} — {row['пакет']}:")
    for u in row['переходы']:
        print(f"  {u}")
    print()

# === Уникальные комбинации для корректной статистики ===
df['уникальная_комбинация'] = df.apply(
    lambda row: f"{row['e-mail_оплаты']} — {row['девайс_оплаты']} — [{row['tracker_name']}] — {row['сумма']}₽", axis=1
)
repeat_df['уникальная_комбинация'] = repeat_df.apply(
    lambda row: f"{row['e-mail_оплаты']} — {row['девайс_оплаты']} — [{row['tracker_name']}] — {row['сумма']}₽", axis=1
)

unique_full = df.drop_duplicates(subset=['уникальная_комбинация'])

# === Статистика по трекерам (FULL) ===
print("\nСтатистика по трекерам (full) по неделям:\n")
weekly_tracker_stats = (
    unique_full.groupby(['год', 'неделя', 'неделя_диапазон', 'tracker_name'])
    .agg(
        сумма=('сумма', 'sum'),
        количество=('уникальная_комбинация', 'count')
    )
    .reset_index()
    .sort_values(by=['год', 'неделя', 'сумма'], ascending=[False, False, False])
)

for _, row in weekly_tracker_stats.iterrows():
    print(f"{row['неделя_диапазон']} — {row['tracker_name']}: {row['количество']} пользователей, {row['сумма']}₽")

Отправка запроса в AppMetrica...
Данные получены.
Переходы по неделям и пакетам:

2025-06-09/2025-06-15 — PREMIER [477]: 5
2025-06-09/2025-06-15 — START [470]: 1
2025-06-09/2025-06-15 — Мегапикс [456]: 30
2025-06-02/2025-06-08 — PREMIER [477]: 12
2025-06-02/2025-06-08 — START [470]: 2
2025-06-02/2025-06-08 — Мегапикс [456]: 17
2025-05-26/2025-06-01 — PREMIER [477]: 2
2025-05-26/2025-06-01 — START [470]: 3
2025-05-26/2025-06-01 — Мегапикс [456]: 20
2025-05-19/2025-05-25 — PREMIER [477]: 3
2025-05-19/2025-05-25 — START [470]: 5
2025-05-19/2025-05-25 — Мегапикс [456]: 16
2025-05-12/2025-05-18 — PREMIER [477]: 4
2025-05-12/2025-05-18 — Мегапикс [456]: 19
2025-05-05/2025-05-11 — PREMIER [477]: 2
2025-05-05/2025-05-11 — START [470]: 3
2025-05-05/2025-05-11 — Мегапикс [456]: 20
2025-04-28/2025-05-04 — PREMIER [477]: 2
2025-04-28/2025-05-04 — START [470]: 2
2025-04-28/2025-05-04 — Мегапикс [456]: 10
2025-04-21/2025-04-27 — PREMIER [477]: 2
2025-04-21/2025-04-27 — Мегапикс [456]: 11
2025-04-14/

# Неделя подсчет повторных

In [7]:
import pandas as pd
import time
from datetime import datetime
import requests
from io import StringIO

# === Пути к файлам ===
PAYMENTS_PATH = 'export_(ReportPayments.2025-02-14 - 2025-06-15.).csv'

# === Авторизация и параметры AppMetrica ===
API_TOKEN = "y0__xDunbWlqveAAhianDcgvtvu8hI4Lgj1FE3Wx6z8be6gSyQ7sTrc4A"
APPLICATION_ID = "4661140"
DATE_SINCE = "2025-02-14 00:00:00"
DATE_UNTIL = datetime.now().strftime("%Y-%m-%d 00:00:00")

# === Классификация переходов ===
def classify_transactions(group):
    group = group.sort_values('дата')
    types = []
    seen_promo = False
    full_found = False

    for _, row in group.iterrows():
        тип = str(row['тип_платежа']).strip().lower()
        is_auto = тип == 'автоплатеж'
        is_primary = тип == 'первичный'
        is_promo = row['промо'] == 1

        if is_promo:
            seen_promo = True
            types.append('promo')
        elif seen_promo and is_auto and not full_found:
            types.append('full')
            full_found = True
        elif not seen_promo and is_primary and not full_found:
            types.append('full')
            full_found = True
        elif full_found and is_auto:
            types.append('repeat')
        else:
            types.append('other')
    
    group['тип_сделки'] = types
    return group


def fetch_installations_csv():
    url = (
        f"https://api.appmetrica.yandex.ru/logs/v1/export/installations.csv?"
        f"application_id={APPLICATION_ID}"
        f"&date_since={DATE_SINCE.replace(' ', '%20')}"
        f"&date_until={DATE_UNTIL.replace(' ', '%20')}"
        f"&date_dimension=default"
        f"&fields=tracker_name,profile_id"
    )
    headers = {"Authorization": f"OAuth {API_TOKEN}"}
    
    print("Отправка запроса в AppMetrica...")
    for attempt in range(30):
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            print("Данные получены.")
            df = pd.read_csv(StringIO(response.text), header=0)
            return df
        elif response.status_code == 202:
            print(f"Попытка {attempt + 1}/30: данные ещё не готовы, ждём ...")
            time.sleep(20)
        else:
            raise Exception(f"Ошибка: {response.status_code}, {response.text}")
    
    raise TimeoutError("Данные не были готовы в течение 5 минут.")


# === Загрузка платежей ===
with open(PAYMENTS_PATH, 'r', encoding='utf-8') as f:
    lines = f.readlines()

header_line = lines[10].strip().split(';')
df = pd.read_csv(PAYMENTS_PATH, skiprows=11, sep=';', names=header_line)
df.columns = [col.strip().lower().replace(' ', '_') for col in df.columns]
df['дата'] = pd.to_datetime(df['дата'], errors='coerce')

# === Фильтрация успешных платежей ===
df = df[
    (df['статус'].str.lower() == 'успешно') & 
    (df['e-mail_оплаты'].notnull())
].sort_values(by=['e-mail_оплаты', 'id_пакета', 'дата'])

df = df.groupby(['e-mail_оплаты', 'id_пакета']).apply(classify_transactions).reset_index(drop=True)

# === Повторные платежи ===
repeat_df = df[df['тип_сделки'] == 'repeat'].copy()
df = df[df['тип_сделки'] == 'full']

# === Загрузка трекеров через API ===
install_df = fetch_installations_csv()
install_df = install_df.dropna(subset=['profile_id'])
install_df = install_df.rename(columns={'profile_id': 'девайс_оплаты'})
install_df['tracker_name'] = install_df['tracker_name'].fillna('unknown')

df = df.merge(install_df, on='девайс_оплаты', how='left')
df['tracker_name'] = df['tracker_name'].fillna('unknown')
repeat_df = repeat_df.merge(install_df, on='девайс_оплаты', how='left')
repeat_df['tracker_name'] = repeat_df['tracker_name'].fillna('unknown')

# === Добавление года и номера недели ===
for target_df in [df, repeat_df]:
    calendar = target_df['дата'].dt.isocalendar()
    target_df['год'] = calendar.year
    target_df['неделя'] = calendar.week
    target_df['неделя_диапазон'] = target_df['дата'].dt.to_period('W').astype(str)

# === Создание строки пользователя ===
df['пользователь'] = df.apply(
    lambda row: f"{row['e-mail_оплаты']} — {row['девайс_оплаты']} — [{row['tracker_name']}] — {row['сумма']}₽", axis=1
)

# === Группировка по неделям и пакетам ===
grouped = (
    df.groupby(['год', 'неделя', 'неделя_диапазон', 'пакет'])
    .agg(переходы=('пользователь', lambda x: sorted(set(x))))
    .reset_index()
    .sort_values(by=['год', 'неделя'], ascending=False)
)

# === Уникальные комбинации для корректной статистики ===
df['уникальная_комбинация'] = df.apply(
    lambda row: f"{row['e-mail_оплаты']} — {row['девайс_оплаты']} — [{row['tracker_name']}] — {row['сумма']}₽", axis=1
)
repeat_df['уникальная_комбинация'] = repeat_df.apply(
    lambda row: f"{row['e-mail_оплаты']} — {row['девайс_оплаты']} — [{row['tracker_name']}] — {row['сумма']}₽", axis=1
)

unique_full = df.drop_duplicates(subset=['уникальная_комбинация'])

# === Статистика по повторным платежам (REPEAT) с учётом уникальности в пределах недели ===
print("\nСтатистика по повторным платежам (repeat) по неделям и трекерам:\n")
unique_repeat = repeat_df.drop_duplicates(
    subset=['год', 'неделя', 'tracker_name', 'уникальная_комбинация']
)

repeat_stats = (
    unique_repeat.groupby(['год', 'неделя', 'неделя_диапазон', 'tracker_name'])
    .agg(
        сумма=('сумма', 'sum'),
        количество=('уникальная_комбинация', 'count')
    )
    .reset_index()
    .sort_values(by=['год', 'неделя', 'сумма'], ascending=[False, False, False])
)

for _, row in repeat_stats.iterrows():
    print(f"{row['неделя_диапазон']} — {row['tracker_name']}: {row['количество']} повторных платежей, {row['сумма']}₽")

# === Повторные платежи по неделям — детальный список ===
print("\nПовторные платежи с разбивкой по неделям:\n")
repeat_df_sorted = repeat_df.sort_values(by=['год', 'неделя', 'дата'])
repeat_grouped = repeat_df_sorted.groupby('неделя_диапазон')

for week_range, group in repeat_grouped:
    count = group.shape[0]
    print(f"{week_range} — [{count}]:")
    for _, row in group.iterrows():
        print(f"{row['e-mail_оплаты']} — {row['девайс_оплаты']} — [{row['tracker_name']}] — {row['сумма']}₽")
    print()

Отправка запроса в AppMetrica...
Данные получены.

Статистика по повторным платежам (repeat) по неделям и трекерам:

2025-06-09/2025-06-15 — PIX_Android_video_app_mid_roll: 15 повторных платежей, 9735.0₽
2025-06-09/2025-06-15 — Google Play: 5 повторных платежей, 2695.0₽
2025-06-09/2025-06-15 — unknown: 2 повторных платежей, 998.0₽
2025-06-09/2025-06-15 — PIX Android push: 1 повторных платежей, 349.0₽
2025-06-09/2025-06-15 — PIX_Direct_10.03.25_Alternative: 1 повторных платежей, 349.0₽
2025-06-02/2025-06-08 — PIX_Direct_10.03.25_Alternative: 3 повторных платежей, 1347.0₽
2025-06-02/2025-06-08 — PIX_Android_video_app_mid_roll: 2 повторных платежей, 1298.0₽
2025-06-02/2025-06-08 — unknown: 1 повторных платежей, 649.0₽
2025-06-02/2025-06-08 — PIX_Android_video_app_pre_roll: 1 повторных платежей, 399.0₽
2025-05-26/2025-06-01 — PIX Android push: 5 повторных платежей, 2995.0₽
2025-05-26/2025-06-01 — PIX_Android_video_app_mid_roll: 4 повторных платежей, 2196.0₽
2025-05-26/2025-06-01 — unknown:

# Табличка с соответствиями для первичных

In [8]:
import pandas as pd
from IPython.display import display

# Пути к файлам
installations_path = "installations.csv"
payments_path = "export_(ReportPayments.2025-06-09 - 2025-06-15.).csv"

# Загрузка CSV
installations_df = pd.read_csv(installations_path)
payments_df = pd.read_csv(payments_path, skiprows=10, sep=';')

# Очистка от пустых profile_id
installations_df_clean = installations_df.dropna(subset=['profile_id'])

# Переименование для объединения
installations_df_clean = installations_df_clean.rename(columns={"profile_id": "Девайс оплаты"})
installations_df_clean = installations_df_clean.drop_duplicates(subset=['Девайс оплаты'])

# Объединение по 'Девайс оплаты'
merged_df = payments_df.merge(installations_df_clean, on="Девайс оплаты", how="inner")

# Сбор нужных столбцов и переименование
result_df = merged_df[["Сумма", "Пакет", "Девайс оплаты", "tracker_name"]]
result_df = result_df.rename(columns={"tracker_name": "Трекер"})

# Вывод таблицы
print("Сводная таблица оплат:")
display(result_df)

# Сумма по трекерам
sum_by_tracker = result_df.groupby("Трекер")["Сумма"].sum().reset_index()
sum_by_tracker = sum_by_tracker.sort_values(by="Сумма", ascending=False)

# Сумма и количество платежей по трекерам
tracker_summary = (
    result_df.groupby("Трекер")
    .agg(
        Сумма=('Сумма', 'sum'),
        Платежи=('Сумма', 'count')
    )
    .reset_index()
    .sort_values(by="Сумма", ascending=False)
)

print("\nСумма и количество платежей по каждому трекеру:")
display(tracker_summary)


Сводная таблица оплат:


,Сумма,Пакет,Девайс оплаты,Трекер
0,499.0,START [470],93360b22addead24,PIX_Android_video_app_mid_roll
1,379.0,Матч! Футбол [498],54f7c434e00ccb61,PIX_Android_video_app_mid_roll
2,349.0,18+ [463],5d0b1e68ea87a7a6,Google Play
3,349.0,18+ [463],1d84f624b7068ae3,Pix_direct_03.06.25
4,379.0,Матч! Футбол [498],1d84f624b7068ae3,Pix_direct_03.06.25
5,199.0,КХЛ [484],1d84f624b7068ae3,Pix_direct_03.06.25
6,299.0,Матч! Премьер [491],1d84f624b7068ae3,Pix_direct_03.06.25
7,349.0,18+ [463],5d0ded4498c61ece,PIX_Android_video_app_pre_roll
8,399.0,PREMIER [477],73ebf8e8ecbab012,PIX_Direct_10.03.25_Alternative
9,199.0,КХЛ [484],950e066ec2e2bdc5,PIX_Android_video_app_mid_roll



Сумма и количество платежей по каждому трекеру:


,Трекер,Сумма,Платежи
1,PIX_Android_video_app_mid_roll,1625.0,5
4,Pix_direct_03.06.25,1226.0,4
3,PIX_Direct_10.03.25_Alternative,399.0,1
0,Google Play,349.0,1
2,PIX_Android_video_app_pre_roll,349.0,1
